## Show tables with Scala and Spark



In [1]:
spark.catalog.listTables.show(false)

+-----------+--------+-----------+---------+-----------+
|name       |database|description|tableType|isTemporary|
+-----------+--------+-----------+---------+-----------+
|stg_commits|default |null       |EXTERNAL |false      |
+-----------+--------+-----------+---------+-----------+



In [2]:
spark.sql("select * from stg_commits").show

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|          repository|          created_at|                 url|                 sha|             node_id|            html_url|        comments_url|              commit|              author|           committer|             parents|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|smart-data-lake/s...|2021-09-12T19:56:34Z|https://api.githu...|04ba7c43ececcfe4d...|MDY6Q29tbWl0MjAxM...|https://github.co...|https://api.githu...|{{zzeekk, zach.ku...|{zzeekk, 13715754...|{zzeekk, 13715754...|[{5aa1029672fd3db...|
|smart-data-lake/s...|2021-09-12T19:55:05Z|https://api.githu...|5aa1

In [3]:
// import smartdatalake
import io.smartdatalake.config.SdlConfigObject.stringToDataObjectId
import io.smartdatalake.config.ConfigToolbox
import io.smartdatalake.workflow.dataobject._
import io.smartdatalake.workflow.ActionPipelineContext
import io.smartdatalake.workflow.action.SDLExecutionId
import io.smartdatalake.app.SmartDataLakeBuilderConfig
import io.smartdatalake.workflow.ExecutionPhase
implicit val ss = spark // make Spark session available implicitly

In [4]:
// read config from mounted directory
val (registry, globalConfig) = ConfigToolbox.loadAndParseConfig(Seq("/mnt/config"), Some(this.getClass.getClassLoader))
// Create the context used by SDL objects
implicit val context = ConfigToolbox.getDefaultActionPipelineContext(spark, registry)

In [5]:
var commits = registry.get[DeltaLakeTableDataObject]("stg-commits").getDataFrame()

In [7]:
commits.printSchema()

root
 |-- repository: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- url: string (nullable = true)
 |-- sha: string (nullable = true)
 |-- node_id: string (nullable = true)
 |-- html_url: string (nullable = true)
 |-- comments_url: string (nullable = true)
 |-- commit: struct (nullable = true)
 |    |-- author: struct (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- email: string (nullable = true)
 |    |    |-- date: string (nullable = true)
 |    |-- committer: struct (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- email: string (nullable = true)
 |    |    |-- date: string (nullable = true)
 |    |-- message: string (nullable = true)
 |    |-- tree: struct (nullable = true)
 |    |    |-- sha: string (nullable = true)
 |    |    |-- url: string (nullable = true)
 |    |-- url: string (nullable = true)
 |    |-- comment_count: long (nullable = true)
 |    |-- verification: struct (nullable = true)
 

In [11]:
commits.where(!($"committer.login" === $"author.login")).
    where(!($"committer.login" === "web-flow")).
    select($"created_at", $"sha", $"committer.login", $"author.login").show()

+--------------------+--------------------+------+-------------+
|          created_at|                 sha| login|        login|
+--------------------+--------------------+------+-------------+
|2021-02-25T11:02:23Z|26d8b74c14825e76d...|zzeekk|MarkusRothSBB|
|2021-02-23T09:48:36Z|3c4fee58421162f6c...|zzeekk|MarkusRothSBB|
|2021-02-23T09:47:15Z|ad7ac5524f5f4133c...|zzeekk|MarkusRothSBB|
+--------------------+--------------------+------+-------------+

